In [102]:
# data wrangling imports
import numpy as np
import pandas as pd 

# other imports
import csv
import re

In [7]:
wolt_df = pd.read_csv("../geotracker/data/wolt.csv")

# creating column to identify the data source
wolt_df["delivery_app"] = "wolt"

In [16]:
# dropping duplicated columns
wolt_df.drop_duplicates(inplace=True)

In [52]:
# preprocessing and turning into float min delivery_fee column
wolt_df.delivery_fee.replace("deliveryMin.", np.NaN, inplace=True)
wolt_df.delivery_fee = wolt_df.delivery_fee.apply(lambda x: 1.90 if x == "1.90Min." else x)

In [135]:
# preprocessing and turning into float min avg_delivery_time column
mask_1 = wolt_df['avg_delivery_time'].notnull()
mask_2 = wolt_df.avg_delivery_time != "Takeaway"
mask = mask_1 & mask_2

wolt_df["min_avg_delivery_time"] = wolt_df[
    mask].loc[:, "avg_delivery_time"].apply(
        lambda x: x.replace("–", " ").split(" ")[0])
wolt_df["max_avg_delivery_time"] = wolt_df[
    mask].loc[:, "avg_delivery_time"].apply(
        lambda x: x.replace("–", " ").split(" ")[1])

wolt_df[["min_avg_delivery_time", "max_avg_delivery_time"
         ]] = wolt_df[["min_avg_delivery_time", "max_avg_delivery_time"]].apply(pd.to_numeric)

# calculating clean avg_delivery_time : avg_delivery_time_clean
wolt_df["avg_delivery_time_clean"] = wolt_df[[
    "min_avg_delivery_time", "max_avg_delivery_time"
]].mean(axis=1)
wolt_df.drop(columns=["avg_delivery_time"], inplace=True)

In [140]:

wolt_df.head()

,restaurant_name,avg_review_score,street,zip_code,city_name,type_of_cuisine,minimum_order_value,delivery_fee,pricyness,latitude,longitude,delivery_app,min_avg_delivery_time,max_avg_delivery_time,avg_delivery_time_clean
0,CRACKBUNS,9.0,Auguststrasse 63,10117,Berlin,"['burger', 'sliders', 'fries']",10.0,NaN,€,52.527203,13.397106,wolt,30.0,40.0,35.0
1,Monsieur Vuong,8.8,Alte Schönhauser Str. 46,10119,Berlin,"['asian', 'pho', 'rice']",10.0,NaN,€€,52.526659,13.407946,wolt,30.0,40.0,35.0
2,Umami Prenzlauer Berg,8.8,Knaackstraße 16-18,10405,Berlin,"['asian', 'vietnamese', 'fish']",10.0,NaN,€€,52.534618,13.419791,wolt,30.0,40.0,35.0
3,Kuchi Mitte,8.8,Gipsstraße 3,10119,Berlin,"['asian', 'japanese', 'sushi']",10.0,NaN,€€,52.527184,13.399308,wolt,30.0,40.0,35.0
4,Kebap with attitude,8.2,Gipsstraße 2,10119,Berlin,"['kebab', 'bowl', 'turkish']",10.0,NaN,€€,52.527243,13.399138,wolt,30.0,40.0,35.0


In [145]:
wolt_df.type_of_cuisine.unique()

array(["['burger', 'sliders', 'fries']", "['asian', 'pho', 'rice']",
       "['asian', 'vietnamese', 'fish']",
       "['asian', 'japanese', 'sushi']", "['kebab', 'bowl', 'turkish']",
       "['hamburger', 'burger', 'meat']", "['dessert', 'donut', 'vegan']",
       "['bowl', 'burrito', 'mexican']",
       "['burger', 'fries', 'milkshake']",
       "['chicken', 'fusion', 'korean']",
       "['burger', 'fries', 'hamburger']",
       "['sushi', 'japanese', 'rice']",
       "['vietnamese', 'asian', 'street food']",
       "['vegan', 'burger', 'fries']", "['american', 'fries', 'potato']",
       "['pizza', 'neapolitan pizza', 'italian']",
       "['chicken', 'korean', 'street food']",
       "['bowl', 'smoothie', 'pasta']", "['greek', 'gyros', 'moussaka']",
       "['asian', 'sushi', 'fish']", "['vietnamese', 'asian']",
       "['fish', 'burger', 'pasta']",
       "['vietnamese', 'asian', 'japanese']",
       "['bap burgers and pastrami', 'hamburger', 'asian']",
       "['italian', 'pasta',